# 📄 DeepLog: Anomaly Detection and Diagnosis from System Logs
> ACM CCS 2017 | Min Du, Feifei Li, Guineng Zheng, Vivek Srikumar  
> [논문 링크](https://users.cs.utah.edu/~lifeifei/papers/deeplog.pdf)

---

## ✅ Day 1 – Abstract & Introduction

---

### 📌 연구 배경과 동기
- **현대 시스템의 로그 문제**  
  대규모 분산 시스템(Hadoop, Spark 등)은 초당 수천 건 이상의 로그를 생성한다. 이 로그는 시스템의 상태와 이상 행동을 파악할 수 있는 핵심 단서이지만, 데이터의 양과 복잡성으로 인해 사람이 직접 분석하기 어렵다.
  

- **기존 접근의 한계**  
  - 규칙 기반 탐지 방식은 미리 정의된 패턴에만 반응하므로 새로운 유형의 이상(New Anomaly)을 탐지하지 못한다.  
  - PCA, SVM 등 통계적·전통적 머신러닝 기법은 로그의 **순차적 의존성(sequence dependency)** 을 반영하지 못한다.  


- **핵심 연구 질문**  
  “로그를 언어 시퀀스로 모델링하여,  
  다음에 발생할 이벤트를 예측함으로써 이상을 탐지할 수 있을까?”
  
<br>

### 📌 핵심 아이디어
- 로그를 **문장(Sequence)** 으로, 각 로그 이벤트를 **단어(Token)** 로 간주한다.  
- LSTM(Long Short-Term Memory) 구조를 이용해 **정상 시퀀스의 다음 이벤트를 예측**한다.  
- 예측 결과와 실제 이벤트가 일치하지 않으면 이를 **이상(Anomaly)** 으로 판단한다.  
- 학습은 정상 로그만으로 이루어지며, **One-class Supervised Learning** 형태로 동작한다.

<br>

### 📌 주요 기여
1. **시퀀스 기반 이상탐지 프레임워크 제안**  
   로그를 시계열 문맥으로 해석하고 LSTM으로 다음 이벤트를 예측하는 방법을 제시했다.  
2. **자동 진단(Diagnosis) 기능 제공** 이상이 발생하면 관련 로그 시퀀스를 역추적하여 원인을 분석할 수 있다.  
3. **성능 향상 입증**  
   PCA, Invariant Mining 등 기존 방식보다 정밀도와 재현율이 모두 향상됨을 보였다.  

<br>

### 📌 핵심 인사이트
- 로그는 단순한 시스템 출력이 아니라, **시스템의 언어**이다.  
- DeepLog는 이 언어의 패턴을 학습하여 시스템의 정상 행동을 이해하고,그로부터 벗어나는 패턴을 이상으로 탐지한다.  
- 이는 규칙을 정의하는 기존 관점에서 벗어나,**정상의 맥락(Context)을 학습하는 관제 방식**으로의 전환을 의미한다.  

<br>

### 🧠 개인적 해석
- DeepLog는 “정상을 정의하면 이상은 자연히 드러난다”는 철학을 잘 보여준다.  
- SOC 환경에서 딥러닝을 적용한 초기 사례로서, **‘로그 = 언어’라는 개념적 전환점**을 마련했다.  
- 이러한 시퀀스 기반 접근은 이후 Transformer 구조를 활용한 LogBERT, LogGPT 등의 후속 연구로 발전하는 출발점이 되었다.

---

## ✅ Day 2 – Related Work & Framework Overview

---

### 📌 관련 연구 (Related Work)

- **규칙 기반 / 통계 기반 접근의 한계**
  - 전통적으로 로그 이상탐지는 **정의된 규칙(rule)**, **정상 분포 기반 모델**, 또는 **PCA 기반 차원축소** 등을 활용했다.  
  - 하지만 이런 방법들은 **패턴 변화에 취약**하고, **로그 간 시간적·의존 관계**를 고려하지 못한다.
  - 예: `Event A → Event B → Event C` 가 정상이라면, `Event A → Event D` 같은 새로운 조합은 감지 불가능하다.


- **머신러닝 기반 접근**
  - SVM, Isolation Forest 등 일부 ML 기법도 사용되었지만,  
    로그가 갖는 **순차적 구조(sequence dependency)** 를 명시적으로 모델링하지 못한다.
  - 결국 로그는 “독립된 점들의 집합”으로 처리되어, 맥락 이해가 불가능했다.


- **DeepLog의 새로운 시도**
  - DeepLog는 로그를 **자연어 문장처럼 모델링**하여,  
    **다음 이벤트 예측(next event prediction)** 문제로 변환한다.  
  - 이를 통해 시간 순서, 이벤트 간 전이 패턴, 반복 구조 등을 학습할 수 있다.
  
<br>

### 📌 전체 구조 (Framework Overview)


#### 1️⃣ 로그 전처리 단계 (Log Parsing)

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
INFO Block 1234 received from node 10.1.1.5  
→ Block <*> received from node <*>  
→ Event ID = 45  
</div>


- 로그는 원래 **비정형 문자열**이므로, 이를 **Log Key** 단위로 변환한다.  
- 위와 같은 과정을 통해 로그를 **토큰화된 시퀀스(event ID sequence)** 로 표현할 수 있다.  
- 일반적으로 **Drain**, **Spell** 같은 로그 파서(parser)를 사용해 정규화한다.  


#### 2️⃣ 시퀀스 모델 학습 (Sequence Modeling)

- DeepLog는 로그 시퀀스를 **LSTM(Long Short-Term Memory)** 기반 모델에 입력하여 다음에 발생할 이벤트를 예측한다.  
- 모델은 최근 **W개의 이벤트 윈도우(window)** 를 관찰하고, 그 문맥으로부터 다음 이벤트를 예측한다.

$$
P(x_{t+1} | x_t, x_{t-1}, \ldots, x_{t-W+1})
$$

- 학습은 **정상 로그 데이터만**을 이용하며,이는 비정상 레이블이 없는 환경에서의 **One-class Supervised Training** 형태이다.  


#### 3️⃣ 이상 탐지 (Anomaly Detection)

- 실제 로그 스트림에서 다음 이벤트가 발생했을 때,  
  모델이 예측한 상위 Top-k 이벤트 목록에 없다면 → **이상(Anomaly)** 으로 판단한다.
  
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
Input sequence: [E1, E2, E3, E4]  
Predicted next events: [E5, E6, E7]  
Actual next event: E9  
→ E9 ∉ Top-k → **Anomaly Detected** 
</div>

- 이런 방식은 새로운 이벤트 조합이나 이전에 없던 패턴도 감지할 수 있게 한다.  


#### 4️⃣ 진단 단계 (Diagnosis)

- 이상이 감지되면, DeepLog는 해당 시퀀스의 **hidden state 변화**를 분석하여 이상이 발생한 위치(원인 이벤트)를 추정한다.  
- 즉, 단순 탐지뿐 아니라 **원인 규명(root-cause analysis)** 도 수행할 수 있다. 

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
예시:  
정상: A → B → C → D  
이상:  A → B → X → D
</div>

→ Hidden state 변화가 X에서 급격함 → X가 root cause로 추정 

- 실제 논문에서는 LSTM의 내부 상태를 시각화하여 정상/비정상 시퀀스의 차이를 명확히 보여주었다.  

<br>

### 📌 핵심 인사이트

- 로그를 **시간 순서가 있는 언어 시퀀스**로 해석한 것이 핵심 혁신이다.  
- LSTM은 시퀀스 내의 **장기 의존성(Long-term dependency)** 을 학습하여 시스템의 “정상 동작 패턴”을 내재적으로 파악한다.  
- 이상 탐지는 더 이상 “규칙 위반”이 아니라, **예측 불일치(prediction inconsistency)** 로 정의된다.  

<br>

### 🧠 개인적 해석

- DeepLog의 파이프라인은 NLP의 “다음 단어 예측(next-token prediction)” 구조와 동일하다.  
- 정상 로그만으로 학습한다는 점에서, **One-Class Learning** 혹은 **Self-Supervised Prediction**의 초기 형태라 할 수 있다.  
- “Top-k 예측 안에 없으면 이상”이라는 간단한 규칙은 실제 시스템 환경에서도 구현이 간단하고 효과적이다.

---

## ✅ Day 3 – Methodology (모델 구조 및 학습 방식)

---

### 📌 전체 구조 개요
DeepLog는 로그 이벤트 시퀀스를 입력받아,  
**이전 로그들의 패턴으로부터 다음 이벤트를 예측**하는 **LSTM 기반 시퀀스 모델**이다.

전체 파이프라인은 다음 3단계로 구성된다.

1. **로그 전처리 (Log Parsing & Mapping)**  
2. **정상 시퀀스 학습 (Model Training)**  
3. **이상 탐지 및 진단 (Anomaly Detection & Diagnosis)**  

---

### 1️⃣ 로그 전처리 (Parsing & Mapping)

- **원본 로그(raw log)** 는 자연어와 유사하지만 불필요한 정보(타임스탬프, ID 등)가 포함되어 있다.  
- 이를 **Log Parser**를 통해 **이벤트 템플릿(event template)** 으로 변환한다.  
  예:  
```bash
Original: "Received block blk_1234 from node 5"
Template: "Received block <> from node <>"
```
- 각 템플릿에 고유한 **Event ID**를 부여한다.  
예:  
```bash
E1: Received block <> from node <>
E2: Sent block report to namenode
```

- 이렇게 변환된 로그 시퀀스는 `[E1, E1, E2, E3, …]` 형태의 **정수 시퀀스**로 표현된다.

---

### 2️⃣ 정상 시퀀스 학습 (LSTM Training)

#### 📌 입력 구조
- 입력: 최근 **`w`개의 이벤트** (sliding window)  
- 출력: 다음 이벤트의 ID  

즉,  
> 입력 `[E1, E2, E3]` → 출력 `[E4]`  

이처럼 로그를 언어 모델처럼 학습한다.

#### 📌 모델 구성
- **LSTM 계층**: 이벤트 간의 순차적 의존성을 학습  
- **Fully Connected Layer**: LSTM의 마지막 hidden state를 이용해 다음 이벤트 확률 분포 예측  
- **Softmax Layer**: 모든 이벤트 ID에 대한 확률을 산출  

#### 📌 손실 함수
- Cross-Entropy Loss  
→ 예측한 확률 분포와 실제 다음 이벤트의 one-hot 벡터 간 차이를 최소화  

---

### 3️⃣ 이상 탐지 (Anomaly Detection)

학습 후, 새로운 로그 시퀀스가 들어오면  
DeepLog는 **다음 이벤트의 확률 상위 K개**를 예측한다.

- 만약 실제 다음 이벤트가 **Top-K 목록 안에 있지 않으면**,  
→ 해당 이벤트를 **이상(Anomaly)** 으로 판단한다.

> 예:  
> 모델이 다음 이벤트로 [E2, E3, E5]를 예측했는데,  
> 실제 로그가 E9라면 → 이상 발생!

이때 K는 **False Positive Rate (FPR)** 과 **Detection Rate** 간의 균형을 맞추기 위해 설정된다.  

---

### 4️⃣ 이상 진단 (Diagnosis)

- 이상이 탐지되면, DeepLog는 그 직전의 **LSTM hidden state sequence**를 분석하여  
어떤 이벤트나 경로가 비정상적인 영향을 주었는지를 추적한다.  
- 즉, 단순히 “이상 있음”이 아니라,  
**“어떤 시퀀스 맥락에서 왜 이상이 발생했는가”** 를 설명할 수 있다.  

---

### 📊 모델 요약 구조

| 단계 | 설명 | 기술 |
|------|------|------|
| Log Parsing | 로그를 이벤트 ID로 변환 | Drain, IPLoM 등 파서 |
| Sequence Modeling | LSTM으로 시퀀스 학습 | 입력 윈도우 크기 w |
| Prediction | 다음 이벤트 확률 예측 | Softmax |
| Detection | Top-K 예측과 실제 비교 | Threshold 기반 |
| Diagnosis | 원인 시퀀스 추적 | Hidden state 분석 |

---

### 💡 핵심 인사이트
- DeepLog는 로그를 언어처럼 다루며,  
“다음 문장을 예측”하듯 “다음 이벤트”를 예측한다.  
- LSTM의 **기억력(memory cell)** 이 과거 이벤트의 맥락을 유지함으로써,  
정상적인 시스템 동작 패턴을 학습한다.  
- 이상 탐지는 결국 “예상치 못한 문장 전개”를 찾아내는 과정이다.  

---

### 🧠 개인적 해석
- DeepLog의 핵심은 **“로그 = 언어 모델링”** 이라는 사고의 전환이다.  
- 로그의 순서(Temporal Context)는 단순 빈도보다 훨씬 중요한 정보임을 보여준다.  
- LSTM이 정상 맥락을 학습하는 방식은 오늘날 **Transformer 기반 로그 이상탐지 (예: LogBERT)** 로 이어진다.  
- 실무에서 SOC 로그나 서버 이벤트 로그에도 동일한 구조를 적용할 수 있다 — 단, Parsing 품질이 전체 성능을 좌우한다.  